In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from glob import glob
import json
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
from pprint import pprint
import seaborn as sns

In [3]:
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)

In [4]:
data_dir = '../data/data_public/B'
build_dir = '../build/'
model_dir = os.path.join(build_dir, 'model')
metric_dir = os.path.join(build_dir, 'metric')
val_dir = os.path.join(build_dir, 'val')
tst_dir = os.path.join(build_dir, 'tst')
sub_dir = os.path.join(build_dir, 'sub')

info_file = os.path.join(data_dir, 'train', 'info.json')
train_feature_file = os.path.join(data_dir, 'train', 'main_train.data')
train_label_file = os.path.join(data_dir, 'train', 'main_train.solution')
test_feature_file = os.path.join(data_dir, 'test', 'main_test.data')
table_files = glob(os.path.join(data_dir, 'train', 'table_*.data'))

In [5]:
table_files

['../data/data_public/B/train/table_1.data',
 '../data/data_public/B/train/table_4.data',
 '../data/data_public/B/train/table_3.data',
 '../data/data_public/B/train/table_2.data']

In [6]:
with open(info_file, 'r') as f:
    info = json.load(f)
    
pprint(info)

{'relations': [{'key': ['c_01'],
                'table_A': 'main',
                'table_B': 'table_1',
                'type': 'many_to_many'},
               {'key': ['c_01'],
                'table_A': 'main',
                'table_B': 'table_2',
                'type': 'many_to_many'},
               {'key': ['c_02'],
                'table_A': 'main',
                'table_B': 'table_3',
                'type': 'many_to_many'},
               {'key': ['c_03'],
                'table_A': 'table_3',
                'table_B': 'table_4',
                'type': 'many_to_one'}],
 'start_time': 1550654179,
 'tables': {'main': {'c_01': 'cat',
                     'c_02': 'cat',
                     'c_1': 'cat',
                     'c_10': 'cat',
                     'c_11': 'cat',
                     'c_12': 'cat',
                     'c_13': 'cat',
                     'c_14': 'cat',
                     'c_15': 'cat',
                     'c_2': 'cat',
                     'c_

In [7]:
trn_feature = pd.read_csv(train_feature_file, sep='\t', parse_dates=['t_01'],
                          date_parser=lambda x: pd.to_datetime(x, unit='ms'))
print(trn_feature.shape)
trn_feature.head()

(1111133, 23)


,n_1,n_2,n_3,n_4,n_5,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,c_14,c_15,t_01,c_02,c_01
0,1.0,1.0,1.0,1.0,1.0,14771401151246747869,15866596883391651220,11791503713415070940,13079277978734158383,3.835861e+18,10820307075201601362,18417620598078758937,758300015379192717,5548418450284150504,NaN,8260837335105015110,1675968519903321981,9893430265068503128,2769572935572985836,3904075253307904652,2018-03-24 19:09:53.767,758300015379192717,7683559224152867638
1,1.0,1.0,1.0,1.0,1.0,6843963171864976598,15866596883391651220,11791503713415070940,13079277978734158383,3.835861e+18,10820307075201601362,18417620598078758937,13100475902651630232,5548418450284150504,NaN,8260837335105015110,3067543872916819730,9893430265068503128,2769572935572985836,3904075253307904652,2018-03-24 19:09:53.763,13100475902651630232,7683559224152867638
2,1.0,1.0,1.0,1.0,1.0,6843963171864976598,15866596883391651220,3413318009442949441,9722599372422752642,3.835861e+18,10820307075201601362,4975860912301333943,13100475902651630232,5548418450284150504,NaN,8260837335105015110,3067543872916819730,9893430265068503128,585566737030769369,3904075253307904652,2018-03-24 19:47:37.558,13100475902651630232,3466625114329849607
3,1.0,1.0,1.0,1.0,1.0,14771401151246747869,15866596883391651220,3413318009442949441,9722599372422752642,3.835861e+18,10820307075201601362,4975860912301333943,14519323142507349436,5548418450284150504,NaN,8260837335105015110,16334284566185484423,9893430265068503128,585566737030769369,3904075253307904652,2018-03-24 19:47:37.547,14519323142507349436,3466625114329849607
4,1.0,1.0,1.0,1.0,1.0,34453685010072459,10330238884931639768,2304265376732037242,3875281194045982152,1.877345e+18,10820307075201601362,1979127182565792769,12181824982323572365,5548418450284150504,11858034859462908913,8260837335105015110,6282077033809040155,9893430265068503128,9126960736538673396,3687556303352301219,2018-03-24 19:12:15.648,12181824982323572365,12808806240266689561


In [8]:
trn_feature.t_01.min(), trn_feature.t_01.max()

(Timestamp('2018-03-24 16:00:00.507000'),
 Timestamp('2018-03-24 22:20:26.800000'))

In [9]:
tst_feature = pd.read_csv(test_feature_file, sep='\t', parse_dates=['t_01'],
                          date_parser=lambda x: pd.to_datetime(x, unit='ms'))
print(tst_feature.shape)
tst_feature.head()

/Users/jeong/.conda/envs/jeong/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1000160, 23)


,n_1,n_2,n_3,n_4,n_5,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,c_14,c_15,t_01,c_02,c_01
0,1.0,1.0,1.0,1.0,1.0,6843963171864976598,15866596883391651220,16339600739432251970,8286680241381910257,3.835861e+18,10820307075201601362,14663497792432641760,13720865199350600718,5548418450284150504,NaN,10518631106994074780,10658893174133127453,9893430265068503128,2691706415317097439,3904075253307904652,2018-03-24 23:51:44.767,13720865199350600718,12702015604063619452
1,1.0,1.0,1.0,1.0,1.0,14771401151246747869,14388459878829169670,13386776437999103576,12574667121135963298,1.877345e+18,15526461888607100114,5683589706357444316,10627370015941795275,5548418450284150504,6987792123181367372,10518631106994074780,1675968519903321981,9893430265068503128,12542092865242546972,12046441164647556966,2018-03-24 23:21:17.763,10627370015941795275,7661777125933877919
2,1.0,1.0,1.0,1.0,1.0,6843963171864976598,14388459878829169670,13386776437999103576,12574667121135963298,1.877345e+18,15526461888607100114,5683589706357444316,2686323527814387074,5548418450284150504,6987792123181367372,10518631106994074780,6282077033809040155,9893430265068503128,12542092865242546972,12046441164647556966,2018-03-24 23:21:17.558,2686323527814387074,7661777125933877919
3,1.0,1.0,1.0,1.0,1.0,34453685010072459,14388459878829169670,13386776437999103576,12574667121135963298,1.877345e+18,15526461888607100114,5683589706357444316,521426872083971872,5548418450284150504,6987792123181367372,10518631106994074780,1675968519903321981,9893430265068503128,12542092865242546972,12046441164647556966,2018-03-24 23:21:17.547,521426872083971872,7661777125933877919
4,1.0,1.0,1.0,1.0,1.0,6843963171864976598,15866596883391651220,7628444360510056009,11688242896569479644,3.835861e+18,10820307075201601362,8733541250368404325,9896419517928559685,5548418450284150504,NaN,8260837335105015110,6282077033809040155,9893430265068503128,18324818564172334259,3904075253307904652,2018-03-24 23:51:22.648,9896419517928559685,4757831576056034990


In [10]:
tst_feature.t_01.min(), tst_feature.t_01.max()

(Timestamp('2018-03-24 22:20:27.500000'),
 Timestamp('2018-03-25 01:05:36.798000'))